####reading json data, cleaning it, apply schema, then load to data lake

In [0]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

In [0]:
dbutils.widgets.text("p_file_date", "")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/paths"

In [0]:
%run "../includes/common_functions"

In [0]:
project_f1_accessKey = dbutils.secrets.get(scope="formual1dlsg2-accesskey-scope", key="project-f1-accessKey")    
spark.conf.set("fs.azure.account.key.formula1dlg2.dfs.core.windows.net", project_f1_accessKey)

In [0]:
constructors_df = spark.read.json(f"{raw_container_folder_path}/{v_file_date}/constructors.json")

In [0]:
constructors_df = constructors_df.drop("url")

In [0]:
from pyspark.sql.types import *
constructors_schema = StructType([StructField("constructor_id", IntegerType()),
                                 StructField("constructor_ref", StringType()),
                                 StructField("name", StringType()),
                                 StructField("nationality", StringType())])
                            

In [0]:
#specify schema to cleaned data frame

constructors_schema_df = spark.createDataFrame(constructors_df.rdd, constructors_schema)

In [0]:
constructors_schema_df.createOrReplaceTempView("constructors_view")


In [0]:
%sql
select * from constructors_view

In [0]:
from pyspark.sql.functions import current_timestamp, lit
constructors_final_df = constructors_schema_df.withColumn("ingestion_date", current_timestamp())\
                                            .withColumn("data_source", lit(v_data_source))\
                                            .withColumn("file_date", lit(v_file_date))

In [0]:
#constructors_final_df.write.mode("overwrite").parquet(f"{processed_container_folder_path}/constructors")

constructors_final_df.write.mode("overwrite").format("delta").saveAsTable("f1_processed_db.constructors")

In [0]:
dbutils.notebook.exit("Success")